In [ ]:
from google.colab import drive


drive.mount("/content/drive")

In [ ]:
!git clone --recurse-submodules https://github.com/chickenta2ta/Yolov5_StrongSORT_OSNet.git
%cd Yolov5_StrongSORT_OSNet
!pip install -r requirements.txt

In [ ]:
!python track.py --yolo-weights "/content/drive/MyDrive/Colab Notebooks/Research/Yield Prediction Using YOLOv5/model/best_20221203.pt" --tracking-method ocsort --source "/content/drive/MyDrive/Colab Notebooks/Research/Raw Data/20221020/14A_1.MOV" --imgsz 1920 --conf-thres 0.8 --save-vid --augment --vid-stride 30

In [ ]:
import numpy as np


d = {}

values = list(d.values())
ndarray = np.array(values)

print(ndarray[ndarray > 3].size)